# Homework 5 - Pandas and SQLite (100 Points)

This assignment focuses on your ability to solve computational problems in both Pandas and SQLite.

## General Instructions:

For **Pandas** questions, you may use standard Python and Pandas commands (joining, merging, math, etc.).  Ensure that the answer is printed in the requested format.  (Note: Your solutions will probably require several lines of Python and Pandas DataFrame manipulation.

For **SQLite** questions, you should execute a single query, which may include subqueries and CTEs as needed.  Ensure that the answer is printed in the requested format.  You may use either the `cursor` or the `fancyCursor`, defined below.

In [37]:
### CELL 0 ###
# Please enter your names here:
# 1.) Sterling Bhollah
# 2.) Dan Rahill
##############
# Note: When submitting your homework, if person #1 submits
# the Notebook, then person #2 should also include a note in
# Sakai that their homework was submitted under #1's name.
# This will help us to make sure that everyone is getting their
# grade and comments in a timely manner!
# NOTE: If you are using a nickname, please include your real name in parenthesis.
# Example: Bubba (Corey) Pennycuff
##############

The following cell will set up the SQLite database (in the variable `db`, with a table called `data`) and a Pandas DataFrame (in a variable called `data`).

In [38]:
import numpy as np
import pandas as pd
import sqlite3
data = pd.read_csv('http://bit.ly/2cLzoxH')

db = sqlite3.connect(':memory:')
data.to_sql('data', db)
cursor = db.cursor()
db.row_factory = sqlite3.Row
fancyCursor = db.cursor()

# Show the contents of the `data` DataFrame.
data

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,1987,9216418.0,Africa,62.351,706.157306
1700,Zimbabwe,1992,10704340.0,Africa,60.377,693.420786
1701,Zimbabwe,1997,11404948.0,Africa,46.809,792.449960
1702,Zimbabwe,2002,11926563.0,Africa,39.989,672.038623


In [39]:
# Show the contents of the SQLite `data` table.
# default cursor

cursor.execute('''
    SELECT *
    FROM data
    LIMIT 5
''')

cursor.fetchall()

[(0, 'Afghanistan', 1952, 8425333.0, 'Asia', 28.801, 779.4453145),
 (1, 'Afghanistan', 1957, 9240934.0, 'Asia', 30.331999999999997, 820.8530296),
 (2, 'Afghanistan', 1962, 10267083.0, 'Asia', 31.997, 853.1007099999999),
 (3, 'Afghanistan', 1967, 11537966.0, 'Asia', 34.02, 836.1971382),
 (4, 'Afghanistan', 1972, 13079460.0, 'Asia', 36.088, 739.9811057999999)]

In [40]:
# Show the contents of the SQLite `data` table.
# "fancy" cursor (using the Row factory)

fancyCursor.execute('''
    SELECT *
    FROM data
    LIMIT 5
''')

[dict(row) for row in fancyCursor.fetchall()]

[{'index': 0,
  'country': 'Afghanistan',
  'year': 1952,
  'pop': 8425333.0,
  'continent': 'Asia',
  'lifeExp': 28.801,
  'gdpPercap': 779.4453145},
 {'index': 1,
  'country': 'Afghanistan',
  'year': 1957,
  'pop': 9240934.0,
  'continent': 'Asia',
  'lifeExp': 30.331999999999997,
  'gdpPercap': 820.8530296},
 {'index': 2,
  'country': 'Afghanistan',
  'year': 1962,
  'pop': 10267083.0,
  'continent': 'Asia',
  'lifeExp': 31.997,
  'gdpPercap': 853.1007099999999},
 {'index': 3,
  'country': 'Afghanistan',
  'year': 1967,
  'pop': 11537966.0,
  'continent': 'Asia',
  'lifeExp': 34.02,
  'gdpPercap': 836.1971382},
 {'index': 4,
  'country': 'Afghanistan',
  'year': 1972,
  'pop': 13079460.0,
  'continent': 'Asia',
  'lifeExp': 36.088,
  'gdpPercap': 739.9811057999999}]

## Example: The Biggest Countries

You goal is to list the largest country on each continent, as well as that country's population for the year 2007.

Notice how the code provides the correct variable values to be used at the end of the cell.

In [41]:
### CELL X ###
# Write your code in this cell.
# Do not remove this comment block.  Write your code after this comment block.
#
# Write you PYTHON solution in this cell.
##############


# Filter for year 2007, and only pull the 3 columns of interest.
populations = data[data["year"] == 2007][["country", "continent", "pop"]]

# Sort by population descending, then group by continent and get the first row from each grouping.
groupings = populations.sort_values('pop', ascending = False).groupby('continent').head(1)

# Use a list comprehension (with the DataFrame .iterrows() method) to
# put the results in the proper format.
records = [(row["continent"], row["country"], row["pop"]) for (id, row) in groupings.iterrows()]


##############
# Do not change the code below this point.
##############
for (continent, country, population) in records:
    print(f"In {continent}, {country} has a population of {population:,.0f}.")

In Asia, China has a population of 1,318,683,096.
In Americas, United States has a population of 301,139,947.
In Africa, Nigeria has a population of 135,031,164.
In Europe, Germany has a population of 82,400,996.
In Oceania, Australia has a population of 20,434,176.


In [42]:
### CELL X ###
# Write your code in this cell.
# Do not remove this comment block.  Write your code after this comment block.
#
# Write you SQLite solution in this cell.
##############


fancyCursor.execute("""
    SELECT d.continent, d.country, d.pop
    FROM data d
    INNER JOIN (
      SELECT continent, MAX(pop) AS maxPop
      FROM data
      WHERE year = 2007
      GROUP BY continent
    ) m
      ON d.continent = m.continent AND d.pop = m.maxPop
    ORDER BY pop DESC """)

records = [(row["continent"], row["country"], row["pop"]) for row in fancyCursor.fetchall()]


##############
# Do not change the code below this point.
##############
for (continent, country, population) in records:
    print(f"In {continent}, {country} has a population of {population:,.0f}.")

In Asia, China has a population of 1,318,683,096.
In Americas, United States has a population of 301,139,947.
In Africa, Nigeria has a population of 135,031,164.
In Europe, Germany has a population of 82,400,996.
In Oceania, Australia has a population of 20,434,176.


## Question 1: Living conditions worsen.  (20 Points)

Which countries have had a decrease in their life expectancy when comparing 1952 against 2007?

`records` will contain a list of country names.

In [43]:
### CELL 1 ###
# Write your code in this cell.
# Do not remove this comment block.  Write your code after this comment block.
#
# Write you PYTHON solution in this cell.
##############


exp1952 = data[data["year"] == 1952][["country", "lifeExp"]]
exp2007 = data[data["year"] == 2007][["country", "lifeExp"]]
exp1952 = exp1952.sort_values('country', ascending = True).set_index('country')
exp2007 = exp2007.sort_values('country', ascending = True).set_index('country')

records = exp1952 > exp2007
records = records[records['lifeExp']]
records = [*records.index.values]

##############
# Do not change the code below this point.
##############
print(f"The countries whose life expectancy has decreased since 1952 are: {records}.")

The countries whose life expectancy has decreased since 1952 are: ['Swaziland', 'Zimbabwe'].


In [44]:
### CELL 2 ###
# Write your code in this cell.
# Do not remove this comment block.  Write your code after this comment block.
#
# Write you SQLite solution in this cell.
##############

fancyCursor.execute("""
    SELECT d.country, d.lifeExp
    FROM data d
    Left JOIN (
      SELECT m.lifeExp, m.country
      FROM data m
      WHERE year = 1952) m
    ON d.country = m.country
    WHERE year = 2007 AND (m.lifeEXP - d.lifeExp > 0)
    """)

records = []

for row in fancyCursor.fetchall():
    records.append(row[0])

##############
# Do not change the code below this point.
##############
print(f"The countries whose life expectancy has decreased since 1952 are: {records}.")

The countries whose life expectancy has decreased since 1952 are: ['Swaziland', 'Zimbabwe'].


## Question 2: Where are they now? (20 Points)

As of the new millenium (or, 2002, the closest year in the dataset), how many countries did each continent comprise?

`records` should be a list of tuples in the form of (`continent`, `count`).

In [45]:
### CELL 3 ###
# Write your code in this cell.
# Do not remove this comment block.  Write your code after this comment block.
#
# Write you PYTHON solution in this cell.
##############

df = data[data["year"] == 2002][["continent","country"]]

conts = df.groupby("continent")
records = []
i = ""
for continent, country in conts:
    i = (continent, len(conts.get_group(continent)))
    records.append(i)

##############
# Do not change the code below this point.
##############
for (continent, count) in records:
    print(f"{continent} has {count} countries.")

Africa has 52 countries.
Americas has 25 countries.
Asia has 33 countries.
Europe has 30 countries.
Oceania has 2 countries.


In [46]:
### CELL 4 ###
# Write your code in this cell.
# Do not remove this comment block.  Write your code after this comment block.
#
# Write you SQLite solution in this cell.
##############


fancyCursor.execute('''
SELECT d.continent, COUNT(country) AS count
FROM data d
WHERE d.year = 2002
GROUP BY continent
''')

records = []
for row in fancyCursor.fetchall():
    records.append((row[0], row[1]))


##############
# Do not change the code below this point.
##############
for (continent, count) in records:
    print(f"{continent} has {count} countries.")

Africa has 52 countries.
Americas has 25 countries.
Asia has 33 countries.
Europe has 30 countries.
Oceania has 2 countries.


## Question 3: Best GDP year of the lowest GDP Country.

Find the country with the lowest GDP in the dataset, and then report what their highest GDP has ever been, and what year that was.

`record` is a dictionary (or similar) container with keys "country", "gdpPercap" and "year".

In [61]:
### CELL 5 ###
# Write your code in this cell.
# Do not remove this comment block.  Write your code after this comment block.
#
# Write you PYTHON solution in this cell.
##############

df = data[["country","gdpPercap", "year"]]
low = df.sort_values('gdpPercap', ascending = False).set_index("country").tail(1)
congo = str(*low.index.values)

df = data[data["country"] == congo][["country","gdpPercap", "year"]]
high = df.sort_values("gdpPercap", ascending = False).head(1)
highGdp = int(high["gdpPercap"].values)
highYear = str(*high["year"].values)

record = {'country': congo, 'gdpPercap': highGdp,'year': highYear}

##############
# Do not change the code below this point.
##############
print(f"{record['country']}'s highest GDP Per Capita was {record['gdpPercap']:,.2f} in {record['year']}.")

Congo Dem. Rep.'s highest GDP Per Capita was 905.00 in 1957.


In [63]:
### CELL 6 ###
# Write your code in this cell.
# Do not remove this comment block.  Write your code after this comment block.
#
# Write you SQLite solution in this cell.
##############


fancyCursor.execute('''

SELECT country, gdpPercap, year
FROM data
WHERE country = (
    SELECT country
    FROM data
    WHERE gdpPercap = (
        SELECT min(gdpPercap)
        FROM data)
)
ORDER BY gdpPercap DESC
LIMIT 1;

''')


record = {'country':row[0], 'gdpPercap':row[1], 'year':row[2]}


##############
# Do not change the code below this point.
##############
print(f"{record['country']}'s highest GDP Per Capita was {record['gdpPercap']:,.2f} in {record['year']}.")

Congo Dem. Rep.'s highest GDP Per Capita was 905.86 in 1957.


## Average Live Expectancy

What is the average life expectancy per continent across all years in the dataset?  Order the results by average life expectancy, descending.

`records` should be a list of tuples in the form (`continent`, `avgLifeExp`).

In [49]:
### CELL 7 ###
# Write your code in this cell.
# Do not remove this comment block.  Write your code after this comment block.
#
# Write you PYTHON solution in this cell.
##############

records = []

df = data[['continent','lifeExp']]
conts = df.groupby('continent')

for cont, lifeExp in conts:
    i = [cont, *conts.get_group(cont).mean()]
    records.append(i)


##############
# Do not change the code below this point.
##############
for (continent, avgLifeExp) in records:
    print(f"{continent} has an average life expectancy of {avgLifeExp:,.2f}.")

Africa has an average life expectancy of 48.87.
Americas has an average life expectancy of 64.66.
Asia has an average life expectancy of 60.06.
Europe has an average life expectancy of 71.90.
Oceania has an average life expectancy of 74.33.


In [50]:
### CELL 8 ###
# Write your code in this cell.
# Do not remove this comment block.  Write your code after this comment block.
#
# Write you SQLite solution in this cell.
##############

fancyCursor.execute("""

    SELECT d.continent, AVG(d.lifeExp) as avg
    FROM data d
    GROUP BY d.continent
""")

records = []

for row in fancyCursor.fetchall():
    records.append([row[0], row[1]])



##############
# Do not change the code below this point.
##############
for (continent, avgLifeExp) in records:
    print(f"{continent} has an average life expectancy of {avgLifeExp:,.2f}.")

Africa has an average life expectancy of 48.87.
Americas has an average life expectancy of 64.66.
Asia has an average life expectancy of 60.06.
Europe has an average life expectancy of 71.90.
Oceania has an average life expectancy of 74.33.


## Question 5: Staying Alive... (20 Points)

Which Country has had the largest percentage increase of life expectancy (`lifeExp`) from 1952 to 2007?

To calculate the percentage increase, consider two numbers, `A` and `B`.  The percentage increase from `A` to `B` is `(B - A) / A * 100`.

Your code should provide values for the variables `country` and `percent`.

In [51]:
### CELL 9 ###
# Write your code in this cell.
# Do not remove this comment block.  Write your code after this comment block.
#
# Write you PYTHON solution in this cell.
##############

exp1952 = data[data["year"] == 1952][["country", "lifeExp"]]
exp2007 = data[data["year"] == 2007][["country", "lifeExp"]]
exp1952 = exp1952.sort_values('country', ascending = True).set_index('country')
exp2007 = exp2007.sort_values('country', ascending = True).set_index('country')

records = ((exp2007 - exp1952)/(exp1952))*100

records = records.sort_values('lifeExp', ascending = False).head(1)


country = str(*records.index.values)
percent = float(*records.values)


##############
# Do not change the code below this point.
##############
print(f"{country} had the largest percent increase in life expectancy, {percent:.1f}%")

Oman had the largest percent increase in life expectancy, 101.3%


In [52]:
### CELL 10 ###
# Write your code in this cell.
# Do not remove this comment block.  Write your code after this comment block.
#
# Write you SQLite solution in this cell.
##############


fancyCursor.execute('''
SELECT d.country,(((d.lifeEXP - m.lifeExp)/ m.lifeExp)*100) AS percent 
FROM data d
INNER JOIN(
    SELECT m.country, m.lifeExp
    FROM data m
    WHERE year = 1952)m
    ON d.country = m.country
WHERE year = 2007
ORDER BY percent DESC
''')


row = fancyCursor.fetchone()
result = {'country': row[0], 'percent': row[1]}

##############
# Do not change the code below this point.
##############
print(f"{result['country']} had the largest percent increase in life expectancy, {result['percent']:.1f}%")

Oman had the largest percent increase in life expectancy, 101.3%
